In [47]:
import pandas as pd
import numpy as np
import re
import re as sub
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE
from sklearn.metrics import silhouette_score

In [22]:

try:
    with open('corpus_all.txt', 'r', encoding='utf-8') as file:
        lines = []  # Empty list to store the lines
        for line in file:
            lines.append(line.strip())  # Append the line to the list, removing any leading/trailing whitespace
    
    with open('stopwords-bn.txt', 'r', encoding='utf-8') as test:
        stopwords_bn = test.readlines()
        # the above stopwords contains newline \n
        stop_bn = []

        for word in stopwords_bn:
            stop_bn.append(word.rstrip("\r\n"))
            
except FileNotFoundError:
    print("File not found. Please check the file path and try again.")

#file_path = 'corpus_all.txt'  # Replace with the actual path to your text file
first_words = []  # Empty list to store the first words

try:
    for line in lines:
        words = line.strip().split()  # Split the line into words
        if words:
            first_word = words[0]  # Get the first word
            first_words.append(first_word)  # Append the first word to the list

except FileNotFoundError:
    print("File not found. Please check the file path and try again.")


# Define the status levels
status_levels = ["happy","angry","fear","disgust","sad","surprise",]

# Initialize a list to store the extracted status levels and sentences
extracted_data = []

# Iterate over the texts
for text in lines:
    # Initialize variables to store the extracted status level and sentence
    status = ""
    sentence = ""

    # Iterate over the status levels
    for level in status_levels:
        if level in text:
            # Split the text based on the status level
            split_text = text.split(level, 1)

            # Extract the status level and sentence
            status = level
            sentence = split_text[1].strip()
            break

    # Append the extracted status level and sentence to the list
    extracted_data.append({"Status": status, "Sentence": sentence})
    
df = pd.DataFrame(extracted_data)

In [51]:

def text_pre_processing(text):
    #remove stop words from the text
    text = ' '.join(word for word in text.split() if word not in stop_bn)
    #remove punctuation from the text
    text = re.sub(r'[^\w\s]', '', text)
    #remove numbers from the text
    text = re.sub(r'[0-9]', '', text)
    #remove extra spaces from the text
    text = re.sub(' +', ' ', text)
    return text

df['Sentence'] = df['Sentence'].apply(text_pre_processing)

In [52]:
X = df['Sentence']
# Map sentiment labels to numerical values
label_map  = {'happy': 0, 'angry': 1, 'fear': 2, 'disgust': 3, 'sad': 4, 'surprise': 5}
y = df['Status'].map(label_map)

X[0]

'বরবর মননয পরধনমনতর গণপরজতনতর বলদশ সরকর মননয পরধনমনতর সবপনর ডজটল বলদশর সবপন বক ললনকর একজন ছতর শত সহসর আশ দশর এক অকততম ভলবস নজক দশর যগয গড তলর এক তবর আকঙকষ ভরত হযছলম নজ হত পরতষঠত দশর অনযতম বশববদযলয যশর বজঞন পরযকত বশববদযলয ডজটল বলদশ গডত ধরণর পরযকত বশববদযলয পরতষঠ নসনদহ দশ পরচলনর বচকষণতর পরচয বহন কর যশর বসর সবপনর পরতষঠন শত শত টক বনযগ দহত উজড কর অতযনত দখর সথ বশববদযলয জবনর সচনলগন অনভব করছ য পওযর আননদ শকষরথদর মনপরন উজড পডলখ কথ বপরত কমলমত শকষরথর সদ দনতপত পরণ নশর হমক শললতহনর গলন ছতরতত বতল হমক নয সমমনত শকষকদর মরত যওয গল কট ফলর হমক পরণ নশর হমক কটকত নযমত ঘটন মননয পরধনমনতর শকষরথর শকষকদর পতমতর মত সমমন একজন চতরথ শরণর করমচরর শকষকদর নযমত ঘটনর শকর ডজটল বলদশর সবপন পক হনদর বহনর মত হন যয চখর ছতর বনর সললতহনর হয পরতবদ বহষকর শকষরথদর ভষ আনদলন মকতযদধর বভষকময গলর কথ সশসনর বলদশ আদরশ বশববদযলয বশববদযলযর পরশসনর পষ সনতরস মদক বযবসয শকষকদর লঞচত কর ছতরদর জবন ছনমন খল ছতরদর লঞচত বরল নজর সথপন সবপনর সনর বল গডর সমপরণ পরপনথ মননয পরধনমনতর শকষক শকষরথর মলত আরতনদ যশরর আকশ বতস পরকমপত বসরর বসর একজন করমচর পরশসনর ছতরছযয অনযয পরতকর আজ

In [53]:

tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust the number of features as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(X).toarray()


In [54]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans

# Assuming you have loaded and preprocessed your data
# X_train_tfidf: TF-IDF features, y: class labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, y, test_size=0.2, random_state=42)

# Oversample the training data using SMOTE
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Train the model
kmeans = KMeans(n_clusters=6, n_init=10, random_state=42).fit(X_train_res)

# Predict the cluster labels for the test data
cluster_labels = kmeans.predict(X_test)

# Evaluate clustering quality using silhouette score
silhouette_avg = silhouette_score(X_test, cluster_labels)
print("Silhouette Score:", silhouette_avg)


Silhouette Score: 0.010051719934341228


In [50]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

# Preprocess the text data
# (Tokenization, stop words removal, stemming, etc.)

# Create TF-IDF features
vectorizer = TfidfVectorizer(max_features=1000)  # Adjust max_features as needed
X = vectorizer.fit_transform(df['Sentence']).toarray()  # Convert to dense array

# Use Gaussian Mixture Models (GMM) for clustering
num_clusters = 6  # Choose the number of clusters
gmm = GaussianMixture(n_components=6, random_state=42)
gmm.fit(X)

# Predict the cluster labels for the data
cluster_labels = gmm.predict(X)

# Evaluate clustering quality using silhouette score
silhouette_avg = silhouette_score(X, cluster_labels)
print("Silhouette Score:", silhouette_avg)


Silhouette Score: -0.037415658556968975
